# Qualitative Evaluation Picker

In this notebook, some instances of the produced results are picked for qualitative evaluation

In [1]:
import pandas as pd

In [2]:
allResults = pd.read_csv('../results/allResults.csv')

In [7]:
# Pick num_samples_per_quartile examples from each quartile based on neural score
num_samples_per_quartile = 2
column_to_sort = 'rug-nlp-nli/flan-base-nli-explanation_neural_score'
# Quartile 1
q1 = allResults[allResults[column_to_sort] <= allResults[column_to_sort].quantile(0.25)]
q1 = q1.sample(n=num_samples_per_quartile)

# Quartile 2
q2 = allResults[(allResults[column_to_sort] > allResults[column_to_sort].quantile(0.25)) & (allResults[column_to_sort] <= allResults[column_to_sort].quantile(0.5))]
q2 = q2.sample(n=num_samples_per_quartile)

# Quartile 3

q3 = allResults[(allResults[column_to_sort] > allResults[column_to_sort].quantile(0.5)) & (allResults[column_to_sort] <= allResults[column_to_sort].quantile(0.75))]
q3 = q3.sample(n=num_samples_per_quartile)

# Quartile 4
q4 = allResults[allResults[column_to_sort] > allResults[column_to_sort].quantile(0.75)]
q4 = q4.sample(n=num_samples_per_quartile)

# Concatenate all the quartiles

qualitativeResults = pd.concat([q1, q2, q3, q4])


In [8]:
# Print the results, based on quartiles
qualitativeResults


,premise,hypothesis,label,explanation_1,explanation_2,explanation_3,rug-nlp-nli/flan-base-nli-explanation_neural_score,rug-nlp-nli/flan-base-nli-explanation_prediction,rug-nlp-nli/flan-base-nli-explanation_prediction.1,rug-nlp-nli/flan-base-nli-explanation_labels,...,rug-nlp-nli/flan-base-nli-label-explanation_rouge_2_2,rug-nlp-nli/flan-base-nli-label-explanation_rouge_2_3,rug-nlp-nli/flan-base-nli-label-explanation_rouge_L_1,rug-nlp-nli/flan-base-nli-label-explanation_rouge_L_2,rug-nlp-nli/flan-base-nli-label-explanation_rouge_L_3,rug-nlp-nli/flan-base-nli-label-explanation_rouge_1_max,rug-nlp-nli/flan-base-nli-label-explanation_rouge_2_max,rug-nlp-nli/flan-base-nli-label-explanation_rouge_L_max,rug-nlp-nli/flan-base-nli-label-explanation_correct_label?,rug-nlp-nli/flan-base-nli-label-explanation_label_difference
2392,A crowd of people in stadium style seating app...,A group of people are excited to be here.,contradiction,The people can not be disinterested and excite...,The people cannot appear disinterested and exc...,The group of people are either disinterested i...,0.059175,Disinterested and excited are different.,Disinterested and excited are different.,['The people can not be disinterested and exci...,...,"Score(precision=0.4, recall=0.2, fmeasure=0.26...","Score(precision=0.0, recall=0.0, fmeasure=0.0)","Score(precision=0.5, recall=0.25, fmeasure=0.3...","Score(precision=0.5, recall=0.2727272727272727...","Score(precision=0.3333333333333333, recall=0.1...",0.352941,0.266667,0.352941,True,"predicted: contradiction, target: contradiction"
3451,2 kids who look related are hanging out with 1...,Two youths were pushing each other.,contradiction,The kids can't be hugging and pushing at the s...,Pushing each other and hugging are opposite ac...,The kids can't be pushing each other while han...,0.039130,Hugging and pushing are different.,Hugging and pushing are different.,"[""The kids can't be hugging and pushing at the...",...,"Score(precision=0.18181818181818182, recall=0....","Score(precision=0.2727272727272727, recall=0.3...","Score(precision=0.3333333333333333, recall=0.3...","Score(precision=0.25, recall=0.375, fmeasure=0.3)","Score(precision=0.4166666666666667, recall=0.4...",0.500000,0.285714,0.434783,True,"predicted: contradiction, target: contradiction"
3220,The young lady is giving the old man a hug.,The young sad lady is giving the old man a hug.,neutral,"The mood of the young lady is not specified, s...","We have no idea if the lady is sad, she could ...",The young lady giving the man a hug may not ne...,0.064956,Just because the lady is giving the old man a ...,Just because the lady is giving the old man a ...,"[""The mood of the young lady is not specified,...",...,"Score(precision=0.04, recall=0.071428571428571...","Score(precision=0.2, recall=0.4166666666666667...","Score(precision=0.23076923076923078, recall=0....","Score(precision=0.15384615384615385, recall=0....","Score(precision=0.38461538461538464, recall=0....",0.512821,0.270270,0.512821,True,"predicted: neutral, target: neutral"
3652,Violin soloists take the stage during the orch...,People are playing violin on stage.,entailment,Violin soloists are referred as people playing...,Violin soloists are people who would be playin...,A violin soloists are people who play the violin.,0.076187,Violin soloists are people.,Violin soloists are people.,['Violin soloists are referred as people playi...,...,"Score(precision=0.75, recall=0.157894736842105...","Score(precision=0.75, recall=0.375, fmeasure=0.5)","Score(precision=0.8, recall=0.4444444444444444...","Score(precision=0.8, recall=0.2, fmeasure=0.32...","Score(precision=0.8, recall=0.4444444444444444...",0.571429,0.500000,0.571429,True,"predicted: entailment, target: entailment"
6635,Birds flying above the rolling waves of the oc...,Birds flying above a body of water.,entailment,An ocean is body of water.,An ocean is a specific body of water.,OCEAN IS A FORM OF BODY OF WATER,0.130198,The ocean is a body of wa